# Capstone Assignment

*This notebook is for Capstone assignment*

## Part One

### Prepocessing

In [1]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
import folium
from bs4 import BeautifulSoup
import lxml

In [2]:
data_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(data_url,"lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XiXaJwpAAEQAAHMlrTEAAAAG","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

*Based on the results above, we can see all of the postcode data we need are under class = "wikitable sortable"*

In [3]:
target_table = soup.find("table", {"class": "wikitable sortable"})
target_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [4]:
print(target_table.tr.text)


Postcode
Borough
Neighbourhood



*Get all info in clean format*

In [5]:
postal_table = ""
for trs in target_table.find_all("tr"):
    row = ""
    for tds in trs.find_all("td"):
        row = row + "," + tds.text
    postal_table = postal_table + row[1:]
    
print(postal_table)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Queen's Park,Not assigned
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James 

*In order to read whole file easily, we hereby create a csv file*

In [6]:
local_file = open("toronto_postal_code.csv","w+b")

local_file.write(bytes(postal_table, encoding = "ascii", errors = "ignore"))

8708

*Using the csv we have created, generate a dataframe*

In [7]:
to_data = pd.read_csv("toronto_postal_code.csv", header = None)
to_data.columns=["Postalcode", "Borough", "Neighbourhood"]
to_data.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
to_data2 = to_data[to_data["Borough"] != "Not assigned"]
to_data2.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
to_data2.loc[to_data2["Neighbourhood"] == "Not assigned", "Neighbourhood"] = to_data2["Borough"]
to_data2.head(10)

D:\Python\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### More than one neighborhood can exist in one postal code area

*According to the requirements, we merge neighbourhoods located in the same Borough (using comma to separate)*

In [10]:
to_data_result = to_data2.groupby(["Postalcode", "Borough"], sort = False).agg( ", ".join)
to_data_result.head(10)

,,Neighbourhood
Postalcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Queen's Park,Queen's Park
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


### Use the .shape method to print the number of rows of your dataframe

In [11]:
to_data_result.reset_index(inplace = True)

In [12]:
to_data_result.shape

(103, 3)

## Part One Ends

---

## Part Two

*Using provided csv file to get geo info*

In [13]:
geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


*Merge two dataframes on "Postalcode", which helps us get all the right info*

In [14]:
geo_data.rename(columns = {"Postal Code": "Postalcode"}, inplace = True)
geo_data_merged = pd.merge(geo_data, to_data_result, on = "Postalcode")
geo_data_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


*Change the format to get a dataframe like the one shown during the assignment instruction*

In [15]:
geo_data_result = geo_data_merged[["Postalcode","Borough","Neighbourhood","Latitude","Longitude"]]
geo_data_result.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
geo_data_result["Borough"].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke', "Queen's Park"], dtype=object)

## Part Two Ends

---

## Part Three

*Let's create a map of Toronto*

In [17]:
address = "Toronto, Ontario, Canada"

geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Toronto are {}, {}.".format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [22]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, borough, neighborhood in zip(geo_data_result["Latitude"], geo_data_result["Longitude"], geo_data_result["Borough"], 
                                           geo_data_result["Neighbourhood"]):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = "orange",
        fill = True,
        fill_color = "#ffb84d",
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  
    
map_toronto

*Define Foursquare Credentials and Version*

In [24]:
CLIENT_ID = "EBHQ1RY4QOLUNPZ11KRPAZLQIRSXFOUOGSLZGWWTN354JFUU" 
CLIENT_SECRET = "ZFWVCYME3KFCV2KXEWCDUWYCMVR0JGJFXXGWOJDJPL00GLKC" 
VERSION = "20200120"

print("Your credentails:")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET:" + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EBHQ1RY4QOLUNPZ11KRPAZLQIRSXFOUOGSLZGWWTN354JFUU
CLIENT_SECRET:ZFWVCYME3KFCV2KXEWCDUWYCMVR0JGJFXXGWOJDJPL00GLKC


*Since I'm pretty interested in venues status in "Downtown Toronto", I hereby scaling my research into this particular area*

In [31]:
dt_to = geo_data_result[geo_data_result["Borough"] == "Downtown Toronto"].reset_index(drop = True)
dt_to.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


*Creating a map for downtown*

In [32]:
address = "Downtown Toronto, Ontario, Canada"

geolocator = Nominatim(user_agent="dt_to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [34]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start = 13)

for lat, lng, label in zip(dt_to["Latitude"], dt_to["Longitude"], dt_to["Neighbourhood"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = "blue",
        fill = True,
        fill_color = "#3186cc",
        fill_opacity = 0.7,
        parse_html = False).add_to(map_downtown)  
    
map_downtown

*Explore the first neighborhood in our dataframe*

In [35]:
dt_to.loc[0, "Neighbourhood"]

'Rosedale'

In [37]:
neighborhood_latitude = dt_to.loc[0, "Latitude"]
neighborhood_longitude = dt_to.loc[0, "Longitude"]

neighborhood_name = dt_to.loc[0, "Neighbourhood"]

print("Latitude and longitude values of {} are {}, {}.".format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


*Let's get the top 50 venues that are in Marble Hill within a radius of 1000 meters*

In [44]:
LIMIT = 50
radius = 1000

url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EBHQ1RY4QOLUNPZ11KRPAZLQIRSXFOUOGSLZGWWTN354JFUU&client_secret=ZFWVCYME3KFCV2KXEWCDUWYCMVR0JGJFXXGWOJDJPL00GLKC&v=20200120&ll=43.6795626,-79.37752940000001&radius=1000&limit=50'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e3099c2760a7f001b3f584c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 43.68856260900001,
    'lng': -79.36510816548741},
   'sw': {'lat': 43.670562590999985, 'lng': -79.38995063451262}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcb343f964a520e32e21e3',
       'name': 'Summerhill Market',
       'location': {'address': '446 Summerhill Ave',
        'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
        'lat': 43.68626482142425,
        'lng': -79.37545823237794,
      

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row["categories"]
    except:
        categories_list = row["venue.categories"]
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]["name"]

*Clean the json and structure it into a pandas dataframe*

In [47]:
from pandas.io.json import json_normalize
venues = results["response"]["groups"][0]["items"]
    
nearby_venues = json_normalize(venues)

filtered_columns = ["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues["venue.categories"] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
2,Black Camel,BBQ Joint,43.677016,-79.389367
3,Pie Squared,Pie Shop,43.672143,-79.377856
4,Tinuno,Filipino Restaurant,43.671281,-79.374920


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


*Create a function to repeat the same process to all the neighborhoods in Downtown*

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
downtown_to_venues = getNearbyVenues(names = dt_to["Neighbourhood"],
                                   latitudes = dt_to["Latitude"],
                                   longitudes = dt_to["Longitude"]
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


In [72]:
print(downtown_to_venues.shape)
downtown_to_venues.head()

(875, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.679563,-79.377529,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
2,Rosedale,43.679563,-79.377529,Black Camel,43.677016,-79.389367,BBQ Joint
3,Rosedale,43.679563,-79.377529,Pie Squared,43.672143,-79.377856,Pie Shop
4,Rosedale,43.679563,-79.377529,Tinuno,43.671281,-79.374920,Filipino Restaurant


In [73]:
downtown_to_venues.groupby("Neighbourhood").count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",36,36,36,36,36,36
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50


In [58]:
print('There are {} uniques categories.'.format(len(downtown_to_venues['Venue Category'].unique())))

There are 174 uniques categories.


### Analyze Each Neighborhood

In [74]:
downtown_onehot = pd.get_dummies(downtown_to_venues[['Venue Category']], prefix="", prefix_sep="")

downtown_onehot['Neighbourhood'] = downtown_to_venues['Neighbourhood'] 

fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theater,Theme Restaurant,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
downtown_onehot.shape

(875, 174)

*Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category*

In [75]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theater,Theme Restaurant,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.04,0.00,0.00,0.00,0.02,0.000000,0.000000,...,0.020000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.00,0.00,0.00
1,Berczy Park,0.000000,0.000000,0.00,0.00,0.02,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.00,0.00,0.00
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.066667,0.066667,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.066667,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,...,0.027778,0.00,0.000000,0.027778,0.00,0.00,0.00,0.00,0.00,0.00
4,Central Bay Street,0.000000,0.000000,0.00,0.00,0.02,0.02,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.00,0.00,0.02
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.00,0.00,0.00,0.02,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.04,0.04,0.02,0.00,0.00
6,Christie,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.02,0.00,0.00
7,Church and Wellesley,0.000000,0.000000,0.00,0.00,0.00,0.02,0.00,0.000000,0.000000,...,0.000000,0.02,0.000000,0.000000,0.00,0.00,0.00,0.00,0.02,0.02
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.04,0.00,0.02,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.02,0.02,0.00,0.00,0.00,0.00
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.04,0.00,0.02,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.02,0.02,0.00,0.00,0.00,0.00


*Print each neighborhood along with the top 5 most common venues*

In [76]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          venue  freq
0          Café  0.08
1    Steakhouse  0.06
2   Coffee Shop  0.06
3  Concert Hall  0.04
4         Hotel  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1          Café  0.06
2      Beer Bar  0.06
3        Bakery  0.04
4  Cocktail Bar  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
             venue  freq
0             Café  0.13
1  Harbor / Marina  0.13
2      Coffee Shop  0.13
3          Airport  0.07
4   Airport Lounge  0.07


----Cabbagetown, St. James Town----
                 venue  freq
0           Restaurant  0.06
1                 Café  0.06
2  Japanese Restaurant  0.06
3            Gastropub  0.06
4                Diner  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.12
1  Japanese Restaurant  0.06
2   Chinese Restaurant  0.04
3             Tea Room  0.04
4       Ice Cream Shop  0.04

*Let's put that into a pandas dataframe*

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Steakhouse,American Restaurant,Hotel,Concert Hall,Seafood Restaurant,Salad Place,Brazilian Restaurant,Breakfast Spot
1,Berczy Park,Coffee Shop,Beer Bar,Café,Hotel,Bakery,Japanese Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Liquor Store
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Harbor / Marina,Café,Airport,Garden,Park,Dog Run,Scenic Lookout,Sculpture Garden,Dance Studio
3,"Cabbagetown, St. James Town",Coffee Shop,Diner,Café,Restaurant,Gastropub,Japanese Restaurant,Park,Pet Store,Performing Arts Venue,Pool
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Ice Cream Shop,Italian Restaurant,Café,Chinese Restaurant,Tea Room,Yoga Studio,Plaza,Bubble Tea Shop


*Cluster Neighborhoods*

In [82]:
from sklearn.cluster import KMeans
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(downtown_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 0, 1, 4, 4, 0, 3, 4, 3, 3])

In [83]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = dt_to

downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Coffee Shop,Park,Grocery Store,Juice Bar,Pie Shop,Playground,Office,Convenience Store,Sandwich Place,Candy Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,4,Coffee Shop,Diner,Café,Restaurant,Gastropub,Japanese Restaurant,Park,Pet Store,Performing Arts Venue,Pool
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Coffee Shop,Gay Bar,Thai Restaurant,Restaurant,Dance Studio,Burger Joint,Men's Store,Gastropub,Gym,Hotel
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Café,Park,Pub,Bakery,Performing Arts Venue,Italian Restaurant,Theater,Mexican Restaurant,Yoga Studio
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,4,Coffee Shop,Cosmetics Shop,Italian Restaurant,Bookstore,Tea Room,Theater,Fast Food Restaurant,Clothing Store,Café,Restaurant


In [85]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], 
                                  downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

*Cluster 1*

In [86]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,0,Café,Restaurant,Coffee Shop,Gastropub,Hotel,Gym,Cocktail Bar,BBQ Joint,Farmers Market,Beer Bar
6,Downtown Toronto,0,Coffee Shop,Beer Bar,Café,Hotel,Bakery,Japanese Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Liquor Store
9,Downtown Toronto,0,Hotel,Aquarium,Park,Coffee Shop,Baseball Stadium,Café,Plaza,Japanese Restaurant,Bubble Tea Shop,Italian Restaurant
12,Downtown Toronto,0,Café,Bakery,Vegetarian / Vegan Restaurant,Bookstore,Park,Thai Restaurant,Japanese Restaurant,Comfort Food Restaurant,Music School,College Gym
13,Downtown Toronto,0,Café,Mexican Restaurant,Coffee Shop,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Burger Joint,Hostel,Organic Grocery
15,Downtown Toronto,0,Café,Beer Bar,Coffee Shop,Farmers Market,Bakery,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Restaurant,Jazz Club


*Cluster 2*

In [87]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Coffee Shop,Harbor / Marina,Café,Airport,Garden,Park,Dog Run,Scenic Lookout,Sculpture Garden,Dance Studio


*Cluster 3*

In [88]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Park,Grocery Store,Juice Bar,Pie Shop,Playground,Office,Convenience Store,Sandwich Place,Candy Store


*Cluster 4*

In [89]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,3,Café,Coffee Shop,Steakhouse,American Restaurant,Hotel,Concert Hall,Seafood Restaurant,Salad Place,Brazilian Restaurant,Breakfast Spot
10,Downtown Toronto,3,Café,Coffee Shop,Hotel,Steakhouse,American Restaurant,Salad Place,Deli / Bodega,Restaurant,Gastropub,Concert Hall
11,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,Hotel,Japanese Restaurant,Deli / Bodega,American Restaurant,Gym,Restaurant,Beer Bar
16,Downtown Toronto,3,Coffee Shop,Café,Gym,Beer Bar,Steakhouse,Seafood Restaurant,Gastropub,Restaurant,Concert Hall,American Restaurant
17,Downtown Toronto,3,Café,Korean Restaurant,Grocery Store,Coffee Shop,Cocktail Bar,Indian Restaurant,Pizza Place,School,Pub,Restaurant


*Cluster 5*

In [90]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Coffee Shop,Diner,Café,Restaurant,Gastropub,Japanese Restaurant,Park,Pet Store,Performing Arts Venue,Pool
2,Downtown Toronto,4,Coffee Shop,Gay Bar,Thai Restaurant,Restaurant,Dance Studio,Burger Joint,Men's Store,Gastropub,Gym,Hotel
3,Downtown Toronto,4,Coffee Shop,Café,Park,Pub,Bakery,Performing Arts Venue,Italian Restaurant,Theater,Mexican Restaurant,Yoga Studio
4,Downtown Toronto,4,Coffee Shop,Cosmetics Shop,Italian Restaurant,Bookstore,Tea Room,Theater,Fast Food Restaurant,Clothing Store,Café,Restaurant
7,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Ice Cream Shop,Italian Restaurant,Café,Chinese Restaurant,Tea Room,Yoga Studio,Plaza,Bubble Tea Shop
18,Downtown Toronto,4,Coffee Shop,Park,Gastropub,Italian Restaurant,Yoga Studio,Dance Studio,Café,Sandwich Place,Salon / Barbershop,Ramen Restaurant


*As we can see from the results showing above, coffee shops & cafe are really popular within Downtown Toronto area.*  
*Meanwhile, each cluster area has their own features.*

## Part Three Ends